In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
customers = pd.read_csv('Customer.csv')
transactions = pd.read_csv('Transactions.csv')
prod_cat_info = pd.read_csv('prod_cat_info.csv')

In [ ]:
transactions.rename(columns={'cust_id': 'customer_Id'}, inplace=True)


In [ ]:
print("Columns in prod_cat_info:", prod_cat_info.columns)
print("Columns in transactions:", transactions.columns)

In [ ]:
transactions = transactions.merge(customers, on='customer_Id', how='left')

In [ ]:
transactions = transactions.merge(prod_cat_info, left_on=['prod_cat_code', 'prod_subcat_code'], right_on=['prod_cat_code', 'prod_subcat_code'], how='left')


In [ ]:
transactions['tran_date'] = pd.to_datetime(transactions['tran_date'], format='%d-%m-%Y')
transactions['DOB'] = pd.to_datetime(transactions['DOB'], format='%d-%m-%Y')

In [ ]:
user_item_matrix = transactions.pivot_table(index='customer_Id', columns='prod_subcat_code', values='total_amt', aggfunc='sum', fill_value=0)


In [ ]:
similarity_matrix = cosine_similarity(user_item_matrix)
similarity_df = pd.DataFrame(similarity_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)


In [ ]:
def recommend_products(customer_id, top_n=5):
    if customer_id not in similarity_df.index:
        return "Customer ID not found."
    
    # Get the top N most similar customers
    similar_customers = similarity_df[customer_id].sort_values(ascending=False).iloc[1:top_n+1]
    recommended_products = []
    
    # Collect products bought by the similar customers
    for similar_customer in similar_customers.index:
        customer_products = transactions[transactions['customer_Id'] == similar_customer]['prod_subcat_code'].unique()
        recommended_products.extend(customer_products)
    
    # Remove duplicates by converting to a set and back to a list
    return list(set(recommended_products))

In [ ]:
customer_id_example = 268408
recommendations = recommend_products(customer_id_example)
print("Recommended Products:", recommendations)